In [4]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [5]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [6]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://ekitan.com/event/station-2590',
   'title': '東京駅周辺のイベント - 駅探',
   'content': '東京駅周辺のイベント - 駅探 駅探 | ekitan *   乗換案内 *   飛行機時刻表 *   飛行機時刻表(国内線) *   飛行機時刻表(国際線)NEW *   高速・夜行バス時刻表 NEW 1.   駅探 3.   東京駅周辺のイベント *   条件 - [x] 無料イベント  - [x] 駐車場あり  - [x] 屋内    *   Image 3: 環境省主催 教職員等環境教育・学習推進リーダー養成研修 実践編 対面研修 東京会場 #### 環境省主催 教職員等環境教育・学習推進リーダー養成研修 実践編 対面研修 東京会場 学校教育×SDGs、環境教育・ESD実践 期間 2025年8月18日(月) 会場 ビジョンセンター東京駅前 * 東京都 中央区 * 東京駅／日本橋駅(東京)／京橋駅(東京)駅 1. *   Image 8: 朝ガツ 東京10社巡り開運ウォーキング-城南4社で運気アップ- #### 朝ガツ 東京10社巡り開運ウォーキング-城南4社で運気アップ- 東京十社巡りウォーキングで開運祈願 期間 2025年5月17日(土)このイベントは終了しました 会場 東京駅丸の内駅前広場（集合） * 東京都 千代田区 * 東京駅／二重橋前駅／大手町駅(東京)駅 *   Image 12 東京メトロ日比谷線 八丁堀駅 徒歩4分 36.6万円(管理費等：14,000円) 2LDK / 74.9m 2 / 12階 / 築18年 Image 13 東京メトロ有楽町線 銀座一丁目駅 徒歩4分 22.5万円(管理費等：13,000円) 1LDK / 49.34m 2 / 13階 / 築21年 Image 14 JR山手線 神田駅 徒歩3分 20.0万円(管理費等：20,000円) 1LDK / 40.58m 2 / 4階 / 築1年 東京駅周辺の物件一覧を見る   Copyright©2025 駅探.',
   'score': 0.7938904,
   'raw_content': None},
  {'url': 'https://www.walk

In [8]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [9]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [19]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    print(tool)
    function_name = tool.function.name
    print(function_name)
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [11]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [17]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。

- 東京都の面積は約2,194平方キロメートルです。
- 沖縄県の面積は約2,271平方キロメートルです。

そのため、沖縄県が東京都よりも広いということになります。


In [20]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
ChatCompletionMessageFunctionToolCall(id='call_L7u8tj3wL32rqQsElzHs1baY', function=Function(arguments='{"question":"東京駅 最近1ヶ月 イベント"}', name='get_search_result'), type='function')
get_search_result
最近1ヶ月以内に行われている東京駅周辺のイベントに関する情報は以下の通りです。

1. **東京駅周辺のイベント**  
   [詳細はこちら](https://www.walkerplus.com/event_list/ar0313/sc309880d/)  
   ・エリア: 東京都、千代田区  
   ・イベント内容: 各種イベントが予定されています。BBQビアガーデンや期間限定のレストランなど、多彩なイベントが展開中です。

2. **ENJOY TOKYO**  
   [詳細はこちら](https://www.enjoytokyo.jp/event/list/sta200101/)  
   ・エリア: 東京都  
   ・内容: 東京駅直結の「XEX TOKYO」で開催されるフードイベントや、さまざまな展示会が行われます。

3. **東京駅イベントカレンダー**  
   [詳細はこちら](https://bestcalendar.jp/events/%E6%9D%B1%E4%BA%AC%E9%A7%85)  
   ・詳細: 東京駅での最新イベント情報が掲載されています。特に、展覧会やシーズンイベントなどがあり、随時更新されています。

4. **東京ステーションギャラリー**  
   [詳細はこちら](https://www.ejrcf.or.jp/gallery/)  
   ・日程: 2025年のイベント情報が掲載されていますが、現在も展示やアートイベントが行われています。

5. **Gransta Tokyoのニュース**  
   [詳細はこちら](https://www.gransta.jp/news/)  
   ・内容: 東京駅の地下街Granstaでの新着情報やイベント情報が更

In [21]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

# メッセージにキャラクター設定を追加
messages.append({"role": "system", "content": "あなたは忍者のキャラクターです。常に忍者と分かる喋り方をして下さい"})

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        for i in range(len(messages)):
            # roleがsystemのメッセージは削除しない
            if messages[i]["role"] == "user" or messages[i]["role"] == "assistant":
                del_message = messages.pop(i)
                break

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:今週の大阪の天気は？'

ChatCompletionMessageFunctionToolCall(id='call_sM7NpKP9jnxJTKVnF46qWwYF', function=Function(arguments='{"question":"今週の大阪の天気"}', name='get_search_result'), type='function')
get_search_result
今週の大阪の天気予報は以下のようになっています。

- **10月13日（金）**: 晴れ時々曇り、最高気温28℃、最低気温19℃
- **10月14日（土）**: 晴れ時々曇り、最高気温27℃、最低気温19℃
- **10月15日（日）**: 曇り時々晴れ、最高気温26℃、最低気温20℃
- **10月16日（月）**: 曇り、最高気温24℃、最低気温18℃
- **10月17日（火）**: 雨、最高気温23℃、最低気温19℃
- **10月18日（水）**: 雨、最高気温22℃、最低気温18℃
- **10月19日（木）**: 曇り、最高気温24℃、最低気温19℃

地域によって天気の変動がありますので、詳細な予報は公式の天気サイトで確認することをお勧めします。たとえば、[こちらのリンク](https://tenki.jp/forecast/6/30/6200/27100/10days.html)で詳しい情報をチェックすることができます。


'質問:2025年8月25日週の大阪の天気は？'

ChatCompletionMessageFunctionToolCall(id='call_3rFgzfpgIzaJH1ZyGsCz0Gud', function=Function(arguments='{"question":"2025年8月25日週の大阪の天気予報"}', name='get_search_result'), type='function')
get_search_result
2025年8月25日週の大阪の天気に関する詳細な予報は現在は提供できません。過去の天気記録や予想に関する情報は限られており、具体的な日付については、188ヤマトで通常何をするのかや、その時期の気象傾向については知識がありますが、正確な天気だけは予測することはできません。

現時点では、以下のリンクで過去の天気データを確認できます：
- [tenki.jpの大阪の過去の天気データ](https://tenki.jp/past/2025/08/weather/6/30/)

その後の詳細については、例えば、前年やそれに近い日付の天気を参考にすることができます。天気予報が近づくにつれ、最新の情報を確認することをお勧めします。


...

こんにちは！今日はどのようにお手伝いできますか？


'質問:東北6県は？'

東北地方は、日本の北東部に位置する地域で、以下の6県から成り立っています。

1. 青森県
2. 岩手県
3. 宮城県
4. 秋田県
5. 山形県
6. 福島県

これらの県は、豊かな自然や歴史的な文化が魅力の地域です。


'質問:宮城県のお土産について検索した結果を教えて'

ChatCompletionMessageFunctionToolCall(id='call_0JvXtqc531Vyp5B5cv1wU7eN', function=Function(arguments='{"question":"宮城県のお土産"}', name='get_search_result'), type='function')
get_search_result
宮城県のお土産についての情報をいくつかご紹介します。

1. **宮城県のお土産ランキング**:
   - 宮城県では、様々な名産品やお土産が人気があります。特に地元の特産品を使用した食品が人気です。たとえば、「牛たん」や「ずんだ餅」などが有名です。 ([詳細](https://mangaroad.jp/?p=946))

2. **おすすめのお土産**:
   - 「萩の月」や「ずんだ餅」、「笹かまぼこ」などが代表的なお土産として挙げられています。これらは観光客にも人気で、訪れる際にはぜひお試しください。 ([参考サイト](https://atlas-log.com/miyagi-souvenir/))

3. **特製の地元産品**:
   - 宮城県の地元産品には、農産物や特製の調味料、菓子などもあります。「金華さば」や「仙台味噌」なども名物として知られています。 ([情報リンク](https://tsplus.asahi.co.jp/articles/gift/75865/))

4. **おすすめの参加型体験**:
   - お皿などの工芸品作り体験や、地元の新鮮な食材を使った料理教室など、体験型のお土産も人気があります。 ([詳細](https://travel.rakuten.co.jp/mytrip/howto/sendai-souvenir))

5. **自宅用にぴったりのお土産**:
   - 自宅で楽しむための宮城県産のお土産も多く、例えば「仙台みりん」や「喜久福のずんだ」といった商品があります。 ([詳細情報](https://www.jalan.net/omiyage/040000/))

これらを参考に、宮城県を訪れた際には、ぜひ魅力的なお土産を選んでみてください。

---ご利用ありがとうございました！---
